In [2]:
!pip install tensorflow

In [8]:
import tensorflow as tf
import numpy as np
import os 

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [9]:
x_train = x_train.reshape(-1, 784).astype("float32") / 255.0
x_test  = x_test.reshape(-1, 784).astype("float32") / 255.0



In [10]:
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test  = tf.keras.utils.to_categorical(y_test, 10)


In [14]:
tf.compat.v1.disable_eager_execution()
tf = tf.compat.v1

In [34]:
def layer(input_, weight_shape, bias_shape, phase_train):
    weight_init = tf.random_normal_initializer(stddev=(1.0 / weight_shape[0])**0.5)
    bias_init = tf.constant_initializer(value=0)

    W = tf.get_variable("W", weight_shape, initializer=weight_init)
    b = tf.get_variable("b", bias_shape, initializer=bias_init)

    logits = tf.matmul(input_, W) + b
    return tf.nn.sigmoid(logits)


In [46]:
def encoder(x, n_code, phase_train):
    with tf.variable_scope("encoder"):
        with tf.variable_scope("layer1"):
            h1 = layer(x, [784, 256], [256], phase_train)

        with tf.variable_scope("layer2"):
            h2 = layer(h1, [256, 128], [128], phase_train)

        with tf.variable_scope("code"):
            code = layer(h2, [128, n_code], [n_code], phase_train)

    return code


In [47]:
def decoder(code, n_code, phase_train):
    with tf.variable_scope("decoder"):
        with tf.variable_scope("hidden_1"):
            l1 = layer(code, [n_code, 128], [128], phase_train)
        with tf.variable_scope("hidden_2"):
            l2 = layer(l1, [128, 256], [256], phase_train)
        with tf.variable_scope("output"):
            output = layer(l2, [256, 784], [784], phase_train)
    return output


In [48]:
def loss_fn(output, x):
    with tf.variable_scope("training"):
        l2 = tf.sqrt(tf.reduce_sum(tf.square(output - x), axis=1))
        train_loss = tf.reduce_mean(l2)
        tf.summary.scalar("train_cost", train_loss)
    return train_loss


In [49]:
def training(cost):
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(cost)
    return train_op


In [50]:
def image_summary(label, tensor):
    reshaped = tf.reshape(tensor, [-1, 28, 28, 1])
    return tf.summary.image(label, reshaped, max_outputs=4)

def evaluate(output, x):
    with tf.variable_scope("validation"):
        in_img = image_summary("input_image", x)
        out_img = image_summary("output_image", output)

        l2 = tf.sqrt(tf.reduce_sum(tf.square(output - x), axis=1))
        val_loss = tf.reduce_mean(l2)

        tf.summary.scalar("val_cost", val_loss)
    return val_loss, in_img, out_img


In [51]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
tf = tf.compat.v1

n_code = 32  # latent space size

tf.reset_default_graph()

with tf.variable_scope("autoencoder_model"):
    x = tf.placeholder(tf.float32, [None, 784])
    phase_train = tf.placeholder(tf.bool)

    code = encoder(x, n_code, phase_train)
    output = decoder(code, n_code, phase_train)

    cost = loss_fn(output, x)
    train_op = training(cost)

    val_loss, in_img, out_img = evaluate(output, x)

summary_op = tf.summary.merge_all()


In [52]:
def evaluate(output, x):
    with tf.variable_scope("validation"):
        in_img = image_summary("input_image", x)
        out_img = image_summary("output_image", output)

        l2 = tf.sqrt(tf.reduce_sum(tf.square(output - x), axis=1))
        val_loss = tf.reduce_mean(l2)
        tf.summary.scalar("val_loss", val_loss)

    return val_loss, in_img, out_img


In [53]:
n_code = 32

tf.reset_default_graph()

with tf.variable_scope("autoencoder_model"):
    x = tf.placeholder(tf.float32, [None, 784])
    phase_train = tf.placeholder(tf.bool)

    code = encoder(x, n_code, phase_train)
    output = decoder(code, n_code, phase_train)

    loss = loss_fn(output, x)
    train_op = training(loss)

    val_loss, in_img, out_img = evaluate(output, x)

summary_op = tf.summary.merge_all()


In [54]:
import os

logdir = "logs/autoencoder"
os.makedirs(logdir, exist_ok=True)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

train_writer = tf.summary.FileWriter(logdir + "/train", sess.graph)
val_writer = tf.summary.FileWriter(logdir + "/val")


I0000 00:00:1769768941.199242  463607 mlir_graph_optimization_pass.cc:437] MLIR V1 optimization pass is not enabled


In [57]:
epochs = 10
batch_size = 128

for epoch in range(epochs):
    avg_loss = 0
    total_batch = int(mnist.train.num_examples / batch_size)

    for i in range(total_batch):
        batch_x, _ = mnist.train.next_batch(batch_size)

        _, l, summary = sess.run(
            [train_op, loss, summary_op],
            feed_dict={x: batch_x, phase_train: True}
        )

        train_writer.add_summary(summary, epoch * total_batch + i)
        avg_loss += l / total_batch

    v_loss, v_summary = sess.run(
        [val_loss, summary_op],
        feed_dict={x: mnist.validation.images, phase_train: False}
    )

    val_writer.add_summary(v_summary, epoch)

    print(f"Epoch {epoch+1}, Train Loss: {avg_loss:.4f}, Val Loss: {v_loss:.4f}")


NameError: name 'mnist' is not defined

In [58]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("data/", one_hot=True)


ModuleNotFoundError: No module named 'tensorflow.examples'